Take Test AIS-Modellframe and add some noise

In [13]:
# import sklearn as sk
import numpy as np
import pandas as pd

known = 9 # 6, 9, 12
# train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}.csv")

#filename = "aisTestFile_500k_kkn"
filename = "aisdk-2023-11-11_266288000_s_1_kkn_Step2b"

#df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_test_"+str(known)+"_0.csv")

df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_train_"+str(known)+"_0.csv")

In [14]:
# df.head()
print(len(df.index))
print(len(df))
df.head()

18
18


,input_0_time,input_0_x,input_0_y,input_1_time,input_1_x,input_1_y,input_2_time,input_2_x,input_2_y,input_3_time,...,input_6_x,input_6_y,input_7_time,input_7_x,input_7_y,input_8_time,input_8_x,input_8_y,output_x,output_y
0,29.0,0.0,0.0,26.0,-1.66980,3.206029,23.0,-4.67544,9.492376,21.0,...,-14.69424,30.174514,6.0,-16.03008,33.003386,3.0,-18.36780,38.535413,-21.03948,43.878846
1,30.0,0.0,0.0,27.0,-2.56036,5.657742,23.0,-3.89620,8.486614,20.0,...,-12.35652,27.031489,6.0,-14.47160,31.934894,4.0,-15.58480,34.449462,-17.81120,39.290008
2,30.0,0.0,0.0,27.0,-2.22640,5.154856,23.0,-4.34148,10.058261,21.0,...,-11.13200,24.328446,7.0,-13.35840,28.917545,3.0,-15.58480,33.443784,-16.58668,35.644041
3,30.0,0.0,0.0,26.0,-1.11320,2.325980,23.0,-3.33960,6.915079,19.0,...,-9.35088,20.242356,6.0,-11.57728,24.328554,3.0,-13.58104,28.414757,-15.25084,32.563832
4,30.0,0.0,0.0,28.0,-1.00188,2.137396,23.0,-3.22828,6.223593,20.0,...,-9.68484,20.368162,7.0,-11.68860,24.140053,3.0,-12.69048,26.026000,-14.47160,29.672166


In [15]:
def generate_noise(df, known, noise):
    df = df.copy()
    i = 0
    while i < (known+1):
        range = np.random.normal(0, noise, len(df.index))

        angles = np.random.uniform(0, 2 * np.pi, len(df.index))

        # range = range.reshape(len(df.index), known+1)
        # angles = angles.reshape(len(df.index), known+1)

        # print(type(range))
        # print(range.shape)

        noiseFrame = pd.DataFrame(range, columns=['range']) 
        angleFrame = pd.DataFrame(angles, columns=['angle'])
        noiseFrame = pd.concat([noiseFrame, angleFrame], axis=1)



        noiseFrame[f"noise-x-{str(i)}"] = noiseFrame['range'] * np.cos(noiseFrame['angle']) # Offset - X
        noiseFrame[f'noise-y-{str(i)}'] = noiseFrame['range'] * np.sin(noiseFrame['angle']) # Offset - Y
        noiseFrame.drop(['range', 'angle'], axis=1, inplace=True)

        if i == 0:
            noiseFrame1 = noiseFrame
        else:
            noiseFrame1 = pd.concat([noiseFrame1, noiseFrame], axis=1)

        i = i + 1 

    #print(noise)
    del noiseFrame
    noiseFrame1.head()

    i = 1
    while i < (known):
        df[f'input_{i}_x'] = df[f'input_{i}_x'] + noiseFrame1[f'noise-x-{i}'] - noiseFrame1['noise-x-0']
        df[f'input_{i}_y'] = df[f'input_{i}_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']
        i = i + 1
    df[f'output_x'] = df['output_x'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-x-0']
    df[f'output_y'] = df['output_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']

    df.head()
    return df


In [16]:
import numpy as np
from pathlib import Path
noise_level = 0, 5, 10
for noise in noise_level:
    #path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv")
    path = Path(f"../workdir/AIS-ModelFrames/{filename}_train_{known}_{noise}.csv")
    if not path.exists():     
        test_model = generate_noise(df, known, noise)
        test_model.to_csv(path, index=False)
        print(f"Done with {known} {noise}")

Done with 9 5
Done with 9 10


In [ ]:
noiseFrame = pd.concat([noiseFrame, angleFrame], axis=1)
noiseFrame.head()

In [ ]:
def add_noise(df, noise_level):
    noisy_df = df.copy()
    for col in ['input_0_x', 'input_0_y', 'input_1_x', 'input_1_y', 'input_2_x', 'input_2_y', 'output_x', 'output_y']:
        noise = np.random.normal(0, noise_level, size=len(df))
        noisy_df[col] += noise
    return noisy_df

noise_level = 0.5  # specify the noise level
noisy_df = add_noise(df, noise_level)
noisy_df.head()

In [31]:
# test.head
# len(test)
# print(test.describe())
# print(test[:10])#["input_0_time"])
print(test[2:3])
#print(test["input_0_time"]-test["input_0_time"])
print(len(test))

   input_0_time  input_0_x  input_0_y  input_1_time  input_1_x  input_1_y  \
2          30.0        0.0        0.0          19.0        0.0        0.5   

   input_2_time  input_2_x  input_2_y  output_x  output_y  
2          11.0       -0.2        0.4      -0.8       0.2  
589


In [57]:
# print(test.columns)
# print(test["input_0_time"])

j = 1
while j < known:
    print(j)
    print(test["input_{j}_time"])
    j += 1  


1


KeyError: 'input_{j}_time'

In [44]:
import numpy as np
def generate_noise(df, noise, known):

    df = df.copy()
    df["src"] = "output"

    scaledNoiseLevel = np.random.normal(0, noise, len(test)*known)
    angles = np.random.uniform(0, 2 * np.pi, len(test)*known)


    print(scaledNoiseLevel)
    print(angles)


    noiseXY = [[0] * 2] * known #* len(test)

    #scaledNoiseLevel = noise #/ (len(test)*known)

    track = 0

    while track < len(test):
        i = 0
        j = 1
        while i < known:
            #angle = np.random.uniform(0, 2 * np.pi)
            # noiseXY[i][0] = scaledNoiseLevel * np.cos(angle) # Offset - X
            # noiseXY[i][1] = scaledNoiseLevel * np.sin(angle) # Offset - Y

            noiseXY[i][0] = scaledNoiseLevel[track*known+i] * np.cos(angles[track*known+i]) # Offset - X
            noiseXY[i][1] = scaledNoiseLevel[track*known+i] * np.sin(angles[track*known+i]) # Offset - Y
            i = i + 1

        while j < known:
            df["input_{j}_x"][track] = df["input_{j}_x"][track] - noiseXY[j][0] - noiseXY[0][0]
            df["input_{j}_y"][track] = df["input_{j}_y"][track] - noiseXY[j][1] - noiseXY[0][0]
            j = j + 1
        
        # i = 1
        # while i < known:
        #     df["input_{i}_x"][track] = df["input_{i}_x"][track] - noiseXY[0][0] + noiseXY[i][0]
        #     df["input_{i}_y"][track] = df["input_{i}_y"][track] - noiseXY[0][1] + noiseXY[i][1]
        #     i = i + 1

        # df["output_x"][track] = df["output_x"][track] - noiseXY[0][0]
        # df["output_y"][track] = df["output_y"][track] - noiseXY[0][1]

    df["output_x"] = df["output_x"] - noiseXY[known-1][0] - noiseXY[0][0]
    df["output_y"] = df["output_y"] - noiseXY[known-1][1] - noiseXY[0][0]

        
    track = track + 1
    return df

# def generate_model_frame(df, knowns):
#     # window_size = knowns + 1
#     # skipped_windows = 0

#     # windows = df.reset_index(drop=False).groupby('track').rolling(window_size)
#     # windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
#     # windows = list(windows)
#     res = Parallel(n_jobs=-1)(delayed(generate_noise)(win) for win in tqdm(windows))
#     #res = (generate_row(win) for win in tqdm(windows))
#     return pd.concat(res, ignore_index=True)


## Reshape

In [11]:
# import sklearn as sk
import numpy as np
import pandas as pd

known = 9 # 6, 9, 12
# train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}.csv")

#filename = "aisTestFile_500k_kkn"
filename = "aisdk-2023-11-11_266288000_s_1_kkn_Step2b"

#df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_test_"+str(known)+"_0.csv")

df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_train_"+str(known)+"_0.csv")

In [12]:
df.head()

,input_0_time,input_0_x,input_0_y,input_1_time,input_1_x,input_1_y,input_2_time,input_2_x,input_2_y,input_3_time,...,input_6_x,input_6_y,input_7_time,input_7_x,input_7_y,input_8_time,input_8_x,input_8_y,output_x,output_y
0,29.0,0.0,0.0,26.0,-1.66980,3.206029,23.0,-4.67544,9.492376,21.0,...,-14.69424,30.174514,6.0,-16.03008,33.003386,3.0,-18.36780,38.535413,-21.03948,43.878846
1,30.0,0.0,0.0,27.0,-2.56036,5.657742,23.0,-3.89620,8.486614,20.0,...,-12.35652,27.031489,6.0,-14.47160,31.934894,4.0,-15.58480,34.449462,-17.81120,39.290008
2,30.0,0.0,0.0,27.0,-2.22640,5.154856,23.0,-4.34148,10.058261,21.0,...,-11.13200,24.328446,7.0,-13.35840,28.917545,3.0,-15.58480,33.443784,-16.58668,35.644041
3,30.0,0.0,0.0,26.0,-1.11320,2.325980,23.0,-3.33960,6.915079,19.0,...,-9.35088,20.242356,6.0,-11.57728,24.328554,3.0,-13.58104,28.414757,-15.25084,32.563832
4,30.0,0.0,0.0,28.0,-1.00188,2.137396,23.0,-3.22828,6.223593,20.0,...,-9.68484,20.368162,7.0,-11.68860,24.140053,3.0,-12.69048,26.026000,-14.47160,29.672166


## All in One

In [1]:
import numpy as np
import pandas as pd


# train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}.csv")

#filename = "aisTestFile_500k_kkn"
#filename = "aisdk-2023-11-11_266288000_s_1_kkn_Step2b"
filename = "aisdk-2023-11-08_1_kkn"
#filename = "aisdk-2023-11-08-s_2_kkn"
#filename = "aisdk-2023-11-08-6xs_2_kkn"

#df = pd.read_csv("../workdir/AIS-ModelFrames/"+filename+"_test_"+str(known)+"_0.csv")



def generate_noise(df, known, noise):
    df = df.copy()
    i = 0
    while i < (known+1):
        range = np.random.normal(0, noise, len(df.index))

        angles = np.random.uniform(0, 2 * np.pi, len(df.index))

        # range = range.reshape(len(df.index), known+1)
        # angles = angles.reshape(len(df.index), known+1)

        # print(type(range))
        # print(range.shape)

        noiseFrame = pd.DataFrame(range, columns=['range']) 
        angleFrame = pd.DataFrame(angles, columns=['angle'])
        noiseFrame = pd.concat([noiseFrame, angleFrame], axis=1)



        noiseFrame[f"noise-x-{str(i)}"] = noiseFrame['range'] * np.cos(noiseFrame['angle']) # Offset - X
        noiseFrame[f'noise-y-{str(i)}'] = noiseFrame['range'] * np.sin(noiseFrame['angle']) # Offset - Y
        noiseFrame.drop(['range', 'angle'], axis=1, inplace=True)

        if i == 0:
            noiseFrame1 = noiseFrame
        else:
            noiseFrame1 = pd.concat([noiseFrame1, noiseFrame], axis=1)

        i = i + 1 

    #print(noise)
    del noiseFrame
    noiseFrame1.head()

    i = 1
    while i < (known):
        df[f'input_{i}_x'] = df[f'input_{i}_x'] + noiseFrame1[f'noise-x-{i}'] - noiseFrame1['noise-x-0']
        df[f'input_{i}_y'] = df[f'input_{i}_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']
        i = i + 1
    df[f'output_x'] = df['output_x'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-x-0']
    df[f'output_y'] = df['output_y'] + noiseFrame1[f'noise-y-{i}'] - noiseFrame1['noise-y-0']

    df.head()
    return df

import numpy as np
from pathlib import Path



noise_level = 0, 5, 10
for known in range(3, 13, 1):
    for noise in noise_level:
        #path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv")
        path = Path(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_{noise}.csv")
        if not path.exists():     
            df = pd.read_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_0.csv")
            test_model = generate_noise(df, known, noise)
            test_model.to_csv(path, index=False)
            print(f"Done with {known} {noise}")

Done with 3 5
Done with 3 10
Done with 4 5
Done with 4 10
Done with 5 5
Done with 5 10
Done with 6 5
Done with 6 10
Done with 7 5
Done with 7 10
Done with 8 5
Done with 8 10
Done with 9 5
Done with 9 10
Done with 10 5
Done with 10 10
Done with 11 5
Done with 11 10
Done with 12 5
Done with 12 10
